In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from langchain_core.messages import HumanMessage

In [ ]:
os.chdir("..")

In [3]:
from src.agent import Agent
from src.tools import python_script, problem_solver
from src.prompts import TASK_IDENTIFIER_PROMPT, TASK_SOLVER_PROMPT

In [4]:
question = """
Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass\
costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?
""".strip()

## Problem breakdown

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")
chain = TASK_IDENTIFIER_PROMPT | llm
response = chain.invoke({"problem":question})
tasks = [r.strip() for r in response.content.split(",")]
tasks

['Determine the cost of the first glass',
 'Calculate the cost of every second glass',
 'Calculate the total number of first glasses and second glasses',
 'Calculate the total cost for first glasses',
 'Calculate the total cost for second glasses',
 'Add the total cost of first glasses and second glasses to find the final amount Kylar needs to pay.']

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini")
chain = TASK_SOLVER_PROMPT | llm

status=""
for task in response.content.split(","):
    response = chain.invoke({"problem":question, "task":task, "status":status}).content
    status += f"{task}: {response}\n"
    print(status)
    print("-"*50)

Determine the cost of the first glass: The cost of the first glass is $5.

--------------------------------------------------
Determine the cost of the first glass: The cost of the first glass is $5.
 Calculate the cost of every second glass: The cost of every second glass is $3.

--------------------------------------------------
Determine the cost of the first glass: The cost of the first glass is $5.
 Calculate the cost of every second glass: The cost of every second glass is $3.
 Calculate the total number of first glasses and second glasses: The total number of first glasses is 8, and the total number of second glasses is 8.

--------------------------------------------------
Determine the cost of the first glass: The cost of the first glass is $5.
 Calculate the cost of every second glass: The cost of every second glass is $3.
 Calculate the total number of first glasses and second glasses: The total number of first glasses is 8, and the total number of second glasses is 8.
 Calc

## Agent

In [5]:
tools = [python_script, problem_solver]
agent = Agent(tools)

await agent.init_graph()

In [6]:
messages = [HumanMessage(content="Hey, how are you doing?")]

In [7]:
thread = {"configurable": {"thread_id": "1"}}
response = await agent.graph.ainvoke({"messages": messages}, config={"configurable": {"thread_id": 1, "recursion_limit": 5}})
print("-"*50)
print(response["messages"][-1].content)

--------------------------------------------------
¡Hola! Estoy aquí para ayudarte. ¿En qué puedo asistirte hoy?


In [8]:
messages = [HumanMessage(content=f"I have a problem that I would like to solve.\nProblem:{question}")]
response = await agent.graph.ainvoke({"messages": messages}, config={"configurable": {"thread_id": 1, "recursion_limit": 5}})
print("-"*50)
print(response["messages"][-1].content)

Calling: {'name': 'problem_solver', 'args': {'question': 'Kylar wants to buy 16 glasses. The first glass costs $5, and every second glass costs only 60% of the price. How much does he need to pay for 16 glasses?'}, 'id': 'call_1lsvQKGiWsfrl4S0Rl1Xss0M', 'type': 'tool_call'}
Back to the model!
--------------------------------------------------
Kylar necesita pagar un total de **$64** por las 16 copas. 

La distribución del costo es la siguiente:
- 8 copas a $5 cada una: \(8 \times 5 = 40\)
- 8 copas a $3 cada una (60% del precio): \(8 \times 3 = 24\)

Sumando ambos costos, \(40 + 24 = 64\). 

Si tienes más preguntas o necesitas ayuda con algo más, ¡dímelo!
